# DST Airlines

[LINK](https://docs.google.com/document/d/1wyo5QOe8UZZE0_fm0PNdc_bTUNqjyrBMHSHwH-JfSEM/edit)

Description détaillée :  
De nos jours, il est possible d’avoir des informations sur les vols dans le monde entier et de traquer en temps réel un avion. Nous pouvons observer ce site en guise d’exemple.  
Le but ici est de s’en approcher le plus possible en passant par des API de différentes compagnies aériennes.

Etape | Description | Modules / Masterclass / Templates | Objectif | Conditions de validation du projet
----- | ----------- | --------------------------------- | -------- | ---------------------------------
1 | Exploration de données non structurées | Passer par l’API de Lufthansa pour récupérer des données sur les vols | 103 - Introduction aux APIs de données | Un fichier explicatif du traitement et des différentes données accessibles (doc / pdf). Un exemple de données collectées.


[Lufthansa Open API Documentation](https://developer.lufthansa.com/docs)

[LH OpenAPI testing](https://developer.lufthansa.com/io-docs)

[Public API Method Details](https://developer.lufthansa.com/docs/read/api_details)

!!! [Un repo GIT englobe déjà les requêtes à l'API, il suffirait de le forker](https://github.com/arossmann/lh_open_api)

In [2]:
from Lhapi import Lhapi as api

test = api()


In [2]:
# REFERENCES
# https://developer.lufthansa.com/docs/read/api_details/Reference_Data
# missing nearest airport

reference_items = ['countries', 'cities', 'airports', 'airlines', 'aircraft']
for ref in reference_items:
    test.request_file(ref + '.json', '/mds-references/' + ref)

In [3]:
# get airports where LH is present
test.request_file('airports_lh.json', '/mds-references/airports?LHoperated=1')

In [8]:
# get lounges from airports where LH is present
import json

def read_file(file_path, word):
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip().startswith('"' + word + '"'):
                res = line.split(':')[1].replace(',', '').replace('"', '').strip()
                yield res


airport_lh_codes = read_file("files/airports_lh.json", "AirportCode")
for code in airport_lh_codes:
    print(test.request_api("/offers/lounges/" + code))


400 https://api.lufthansa.com/v1/offers/lounges/ AAA
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAB
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAC
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAD
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAE
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAF
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAG
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAH
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAI
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAJ
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAK
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAL
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAM
?limit=100
None
400 https://api.lufthansa.com/v1/offers/lounges/ AAN
?limit=100
None
400 https://api.lufthansa.com/v1/o

In [ ]:
# OFFERS

